First we import the libraries.

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import statistics as cal
import torch
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, plot_confusion_matrix, confusion_matrix

In [12]:
#load in df
Player = 15
Game = 2

df = pd.read_csv('matrix_Player_' + str(Player) + '_game_' + str(Game) + '_Processed_Action.csv')
# Fill NaN with 0
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.fillna(0)

#Convert Sprinting to 1
df.Action[df.Action == "Sprinting"] = 1

#Delete first 99 rows, so df starts at timeLine == 1.00
df = df.iloc[99: , :]
print(df.iloc[200:500 , :])

     Unnamed: 0     frAcc   frRoAcc   frDispl  frRoAng   frSpeed  timeLine  \
299         299  0.002430 -0.000160 -0.090679 -0.21044 -0.000124      3.00   
300         300 -0.014385 -0.000161 -0.092929 -0.23955 -0.000099      3.01   
301         301 -0.001402 -0.000162 -0.095673 -0.30925 -0.000243      3.02   
302         302 -0.007637 -0.000165 -0.098779 -0.31193 -0.000257      3.03   
303         303  0.026330 -0.000168 -0.102200 -0.37210 -0.000334      3.04   
..          ...       ...       ...       ...      ...       ...       ...   
594         594 -0.016101 -0.001562 -0.855020 -0.24591 -0.000309      5.95   
595         595 -0.033849 -0.001566 -0.857470 -0.24295 -0.000470      5.96   
596         596  0.047428 -0.001572 -0.860010 -0.26550 -0.000809      5.97   
597         597 -0.038892 -0.001578 -0.862210 -0.17479 -0.000334      5.98   
598         598 -0.008833 -0.001584 -0.864330 -0.24918 -0.000723      5.99   

     frameRotationalSpeedX  frameRotationalSpeedY  frameRotatio

/tmp/ipykernel_4455/2176787522.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Action[df.Action == "Sprinting"] = 1


In [13]:
#Convert data into chunks of a second
n = 100 #chunk size (100 datapoints in one second)
chunks = [df[i:i+n] for i in range(0,df.shape[0],n)]
#Set chunks into dataframe
#for chunk in chunks:
frames = []
for chunk in chunks:
    P = pd.DataFrame({'frAcc':[max(chunk['frAcc'].tolist())], 
                      'frRoAcc':[chunk['frRoAcc'].tolist()], 'frDispl':[chunk['frDispl'].tolist()], 
                      'frRoAng':[chunk['frRoAng'].tolist()], 'frSpeed':[max(chunk['frSpeed'].tolist())], 
                      'timeLine':[max(chunk['timeLine'].tolist())], 'frameRotationalSpeedX':[chunk['frameRotationalSpeedX'].tolist()], 
                      'frameRotationalSpeedY':[cal.mean(chunk['frameRotationalSpeedY'].tolist())], 'frameRotationalSpeedZ':[cal.mean(chunk['frameRotationalSpeedZ'].tolist())],
                      'wheelRotationalSpeedX':[max(chunk['wheelRotationalSpeedX'].tolist())], 'wheelRotationalSpeedY':[chunk['wheelRotationalSpeedY'].tolist()],
                      'wheelRotationalSpeedZ':[chunk['wheelRotationalSpeedZ'].tolist()], 'frRoSpeed':[max(chunk['frRoSpeed'].tolist())],
                      'Sum_WheelX_FrameZ':[max(chunk['Sum_WheelX_FrameZ'].tolist())], 'Div_FrameZ_WheelX':[cal.mean(chunk['Div_FrameZ_WheelX'].tolist())],
                      'Filt_WheelX':[cal.mean(chunk['Filt_WheelX'].tolist())], 'Filt_FrameZ':[cal.mean(chunk['Filt_FrameZ'].tolist())],
                      'Action':[max(chunk['Action'].tolist())]})
    frames.append(P)
df = pd.concat(frames,ignore_index=True)

In [33]:
# split chunk data into train test validate (with colum [sum and Div] as input, and action as output)
X_train, X_valid, y_train, y_valid = train_test_split(df[[
                                                        'Sum_WheelX_FrameZ',
                                                        'Div_FrameZ_WheelX',
                                                        'wheelRotationalSpeedX',
                                                        'Filt_FrameZ',
                                                        'Filt_WheelX',
                                                        'frameRotationalSpeedY'
                                                       ]], 
                                                    df.Action.to_numpy().reshape(-1,1), test_size = 0.2, random_state=42)

In [20]:
degree = 2
poly = PolynomialFeatures(degree=degree, include_bias=False)
X_train = poly.fit_transform(X_train)
X_valid = poly.transform(X_valid)

In [22]:
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_valid = X_scaler.transform(X_valid)

In [34]:
y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train)
y_valid = y_scaler.transform(y_valid)

In [19]:
train_ds = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
valid_ds = TensorDataset(torch.tensor(X_valid), torch.tensor(y_valid))

NameError: name 'TensorDataset' is not defined